In [14]:
#Import the necessary packages

from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked

import warnings
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [18]:
#Set the home path

%env HOME=C:\Users\USER\Documents\GitHub\HTML-building
homedir = os.getenv("HOME")

env: HOME=C:\Users\USER\Documents\GitHub\HTML-building


In [3]:
import numpy as np
from matplotlib import pyplot as plt
import os
from astropy.table import Table
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output


In [16]:
    def compile_LS_cutouts(self):
        
        pixscale=1   #standard
        pscale=np.abs(float(self.wise_header['CD1_1']))   #grab transformation matrix of WISE image
        xsize=np.abs(int(self.wise_header['NAXIS1']))   #grab length of WISE image
        xsize_arcsec=pscale*3600*xsize   #length convert to arcseconds
        imsize=int(xsize_arcsec/pixscale)   #convert length to an integer
        imsize=str(imsize)   #convert integer length to a...string
        
        self.filename_LS = self.LS_cutouts+self.VFID+'-LS.jpg'
        
        image_url = f'https://www.legacysurvey.org/viewer/cutout.jpg?ra={self.RA}&dec={self.DEC}&layer=ls-dr9&size={imsize}&pixscale={1}'
        if os.path.exists(self.filename_LS):
            os.remove(self.filename_LS)
            image = wget.download(image_url,out=self.filename_LS)
        else:
            image = wget.download(image_url,out=self.filename_LS)
    
    def create_LS_mosaics(self):
        
        #first must be sure that r-band is projected onto w3 coordinates (ensures all images have same size)
        #r_scaled, footprint = reproject_interp((self.r_im, self.r_header), self.wise_header)

        titles = ['W3 Image', 'W1 Image', 'LS Image']
        images = [self.wise_im, self.w1_im, self.filename_LS]
        
        try:
            bool_masks = [self.w3_mask_bool, self.w1_mask_bool, None]
        except:
            bool_masks = [np.zeros(self.wise_im.shape)+1, 
                          np.zeros(self.w1_im.shape)+1, None]
            print(f'{self.VFID} has no mask images.')
        
        plt.figure(figsize=(12,6))
        for i,im in enumerate(images):
            plt.xlabel('RA')
            if (i==0)|(i==1):
                if i==0:
                    plt.subplot(1,len(images),i+1,projection = self.wcs_w3)
                if i==1:
                    plt.subplot(1,len(images),i+1,projection = self.wcs_w1)
                try:
                    norm = simple_norm(images[i]*bool_masks[i],stretch='asinh',max_percent=99.9)
                    plt.imshow(images[i],origin='lower',cmap='viridis',norm=norm)
                except:
                    norm = simple_norm(self.w1_im,stretch='asinh',max_percent=99.9)
                    plt.imshow(self.w1_im,origin='lower',cmap='viridis',norm=norm)
                plt.ylabel('DEC')
                ax = plt.gca()
                ax.set_yticks([])
            
            if i==2:
                plt.subplot(1,len(images),i+1)
                plt.imshow(mpimg.imread(images[i]),origin='lower')
                plt.gca().invert_yaxis()
                plt.axis("off")

            plt.subplots_adjust(wspace=0,hspace=0)
            plt.title(titles[i],fontsize=24)
        
        if os.path.exists(self.LS_mosaics+self.VFID+'-LS-mosaic.png'):
            os.remove(self.LS_mosaics+self.VFID+'-LS-mosaic.png')
            plt.savefig(self.LS_mosaics+self.VFID+'-LS-mosaic.png',bbox_inches = 'tight',pad_inches=0.2)
        else:
            plt.savefig(self.LS_mosaics+self.VFID+'-LS-mosaic.png',bbox_inches='tight',pad_inches=0.2)
        plt.close()

In [13]:
# Open the HTML file in write mode
with open("output.html", "w") as html:
    # Write the HTML code line by line
    html.write('<html><body>\n')
    html.write('<title>Virgo WISESize Project</title>\n')
    html.write('<style type="text/css">\n')
    html.write('table, td, th {padding: 5px; text-align: center; border: 2px solid black;}\n')
    html.write('p {display: inline-block;;}\n')
    html.write('</style>\n')
    html.write('<table><tr><th>VFID</th><th>Legacy image</th><th>Prefix</th><th>RA</th><th>DEC</th>\n')
    html.write('</tr></table>\n')
    html.write('<br /><br />\n')
    html.write('</html></body>\n')
    html.close()
print("HTML code has been written to output.html")


HTML code has been written to output.html


In [ ]:
Current task : 
    get all of the fits images from the HST first
    Understanding the wise_header and how to link it to my images